# 导入bit文件

In [77]:
import math
from pynq import Overlay
overlay = Overlay("../overlay/cordic.bit")
cordic = overlay.cordiccart2pol_0
x=math.sin(math.pi/3)
y=math.cos(math.pi/3)

## 查看寄存器 

In [78]:
cordic.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  x = Register(x=write-only),
  y = Register(y=write-only),
  r = Register(r=0),
  r_ctrl = Register(r_ap_vld=0, RESERVED=0),
  theta = Register(theta=0),
  theta_ctrl = Register(theta_ap_vld=0, RESERVED=0)
}

# 输入要计算的x值与y值

In [87]:
import struct

x_input=1;
y_input=1.732;
def floatToBytes(f):
    bs = struct.pack("f",f)
    return (bs[3]<<24)+(bs[2]<<16)+(bs[1]<<8)+bs[0]

x=floatToBytes(x_input)  #将输入浮点数转换为二进制
y=floatToBytes(y_input)  
cordic.write(0x10,x)     #写入相应的寄存器中
cordic.write(0x18,y)

### 开始执行

In [88]:
import time

cordic.register_map.CTRL.AP_START = 1

start_time = time.time()

while True:
    reg = cordic.register_map.CTRL.AP_START
    if reg != 1:
        break

end_time = time.time()

print("耗时：{}s".format(end_time - start_time))

耗时：0.0008895397186279297s


### 读出坐标的模值

In [89]:
r=cordic.read(0x20)

def bytesToFloat(h1,h2,h3,h4):
    ba = bytearray()
    ba.append(h1)
    ba.append(h2)
    ba.append(h3)
    ba.append(h4)
    return struct.unpack("!f",ba)[0]

h1=(r&0xff000000)>>24
h2=(r&0x00ff0000)>>16
h3=(r&0x0000ff00)>>8
h4=(r&0x000000ff)
print("模值为")
print(bytesToFloat(h1,h2,h3,h4))  ##将二进制转换为浮点数

模值为
1.9999561309814453


### 读出坐标的角度

In [90]:
theta=cordic.read(0x30)
h1=(theta&0xff000000)>>24
h2=(theta&0x00ff0000)>>16
h3=(theta&0x0000ff00)>>8
h4=(theta&0x000000ff)
print("角度为")
print(bytesToFloat(h1,h2,h3,h4)/math.pi*180)

角度为
59.9991615563842
